In [1]:
import pandas as pd
from collections import defaultdict
import hashlib

def read():
    dfa = pd.read_csv("../data/input/merged_profiles.csv")
    dfb = pd.read_csv("../data/input/personnel.csv")
    return dfa, dfb

dfa, dfb = read()

dfa["source"] = "wci"
dfa["agency"] = "n/a"

dfb["source"] = "llead"
dfb["officer_role"] ="n/a"
dfb["officer_context"] = "n/a"
dfb = dfb[dfb.agency.str.contains("orleans-pd|orleans-so")]

def create_hash_uid(row):
    # Concatenate relevant fields to create a unique string
    unique_string = f"{row['first_name1']}|{row['last_name1']}|{row['first_name2']}|{row['last_name2']}|{row['source1']}|{row['source2']}"
    # Create a hash of the unique string
    return hashlib.md5(unique_string.encode()).hexdigest()

dfa = dfa.rename(columns={"person_uid":  "wcoi_uid"})

dfb = dfb.rename(columns={"uid": "llead_uid"})



df = pd.concat([dfa, dfb])


df.loc[:, "first_name"] = df.first_name.str.lower().str.strip()
df.loc[:, "last_name"] = df.last_name.str.lower().str.strip()

df.loc[:, "fc"] = df.first_name.fillna("").map(lambda x: x[:1])
df.loc[:, "lc"] = df.last_name.fillna("").map(lambda x: x[:1])

df = df[["first_name", "last_name", "fc", "lc", "source", "wcoi_uid", "llead_uid", "agency"]]

print(f"DF SHAPE BEFORE {df.shape}")

df = df.drop_duplicates(subset=["wcoi_uid", "llead_uid"])

print(f"DF SHAPE AFTER {df.shape}")

df = df.reset_index(drop=True)

df.loc[:, "full_name"] = df.first_name.str.cat(df.last_name, sep=" ")

df = df[~((df.full_name.fillna("") == ""))]

full_names = df.first_name.str.cat(df.last_name, sep=" ")


def jaro_winkler_similarity(s1, s2):
    # Jaro-Winkler similarity implementation
    if not s1 or not s2:
        return 0.0

    # Find matching characters
    match_distance = (max(len(s1), len(s2)) // 2) - 1
    s1_matches = [0] * len(s1)
    s2_matches = [0] * len(s2)
    matches = 0
    transpositions = 0

    for i, ch in enumerate(s1):
        start = max(0, i - match_distance)
        end = min(i + match_distance + 1, len(s2))
        for j in range(start, end):
            if s2[j] == ch and s2_matches[j] == 0:
                s1_matches[i] = 1
                s2_matches[j] = 1
                matches += 1
                break

    if matches == 0:
        return 0.0

    # Count transpositions
    k = 0
    for i, ch in enumerate(s1):
        if s1_matches[i]:
            while s2_matches[k] == 0:
                k += 1
            if s1[i] != s2[k]:
                transpositions += 1
            k += 1

    # Calculate Jaro similarity
    jaro = ((matches / len(s1)) + (matches / len(s2)) + ((matches - transpositions / 2) / matches)) / 3

    # Calculate common prefix
    prefix = 0
    for i in range(min(len(s1), len(s2))):
        if s1[i] == s2[i]:
            prefix += 1
        else:
            break
    prefix = min(4, prefix)

    # Calculate Jaro-Winkler similarity
    jaro_winkler = jaro + (0.1 * prefix * (1 - jaro))

    return jaro_winkler

def custom_matcher(df):
    results = []
    pair_idx = 0
    
    groups = defaultdict(list)
    for _, row in df.iterrows():
        key = (row['fc'], row['lc'])
        groups[key].append(row)
    
    for group in groups.values():
        for i, row1 in enumerate(group):
            for j, row2 in enumerate(group[i+1:], start=i+1):
                if row1['source'] == row2['source']:
                    continue
                
                first_name_sim = jaro_winkler_similarity(row1['first_name'], row2['first_name'])
                last_name_sim = jaro_winkler_similarity(row1['last_name'], row2['last_name'])
                sim_score = (first_name_sim + last_name_sim) / 2
                
                if sim_score >= 0.9:
                    results.append({
                        'pair_idx': pair_idx,
                        'sim_score': sim_score,
                        'row_key1': row1.name,
                        'first_name1': row1['first_name'],
                        'last_name1': row1['last_name'],
                        'fc1': row1['fc'],
                        'source1': row1['source'],
                        'agency1': row1['agency'],
                        'wcoi_uid1': row1['wcoi_uid'],
                        'llead_uid1': row1['llead_uid'],
                        'row_key2': row2.name,
                        'first_name2': row2['first_name'],
                        'last_name2': row2['last_name'],
                        'fc2': row2['fc'],
                        'source2': row2['source'],
                        'agency2': row2['agency'],
                        'wcoi_uid2': row2['wcoi_uid'],
                        'llead_uid2': row2['llead_uid']
                    })
                    pair_idx += 1
    
    result_df = pd.DataFrame(results)
    result_df = result_df.sort_values('sim_score', ascending=False)
    
    return result_df


df = custom_matcher(df)

df['person_uid'] = df.apply(create_hash_uid, axis=1)

DF SHAPE BEFORE (12859, 8)
DF SHAPE AFTER (12859, 8)


In [2]:
df

,pair_idx,sim_score,row_key1,first_name1,last_name1,fc1,source1,agency1,wcoi_uid1,llead_uid1,row_key2,first_name2,last_name2,fc2,source2,agency2,wcoi_uid2,llead_uid2,person_uid
54,54,1.000000,1444,michael,buras,m,wci,n/a,3.559646e+09,NaN,2058,michael,buras,m,llead,new-orleans-pd,NaN,0af6f73ba2d8d3d05f50da31fb28079c,36712f4e31b9b7e24db2ef3bf4a46b97
47,47,1.000000,1323,justin,crespo,j,wci,n/a,2.814047e+09,NaN,11108,justin,crespo,j,llead,new-orleans-pd,NaN,d422e81d64416c4ef33e1d44d50e6433,fa02bde7fb28e795208df2dd5d3ca220
84,84,1.000000,1422,steven,nicholas,s,wci,n/a,5.158549e+09,NaN,11526,steven,nicholas,s,llead,new-orleans-pd,NaN,deb8d01f4ce9960b4a2a55ff3c817a1e,218992046dcee921353bae3c123e8386
34,34,1.000000,1407,david,slicho,d,wci,n/a,9.179984e+09,NaN,7884,david,slicho,d,llead,new-orleans-pd,NaN,86b9df7689e1bbd8917690fa80311c88,32f6549a582bea3e53ff8abfeceebcd2
83,83,1.000000,1406,robert,jennewine,r,wci,n/a,1.297401e+09,NaN,12328,robert,jennewine,r,llead,new-orleans-pd,NaN,f3561f60f4e912980f1fc16159d9692e,10d434c6993d474efc8e03f60f57721b
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13,13,0.911111,1499,michael,sauter,m,wci,n/a,5.971844e+09,NaN,10320,michael,sarver,m,llead,new-orleans-pd,NaN,c0075476333d05414f2f6a9369cd0a73,b86241dd37b2da1c58e5da2576021701
11,11,0.911111,1499,michael,sauter,m,wci,n/a,5.971844e+09,NaN,6386,michael,sarver,m,llead,new-orleans-pd,NaN,642c4fd96cae71619898ab6a00d1ae9f,b86241dd37b2da1c58e5da2576021701
26,26,0.906667,1515,sarah,johnson,s,wci,n/a,8.966092e+09,NaN,11775,samantha,johnson,s,llead,new-orleans-pd,NaN,e4a665747962f3ce15d762e5c145b61a,1bbfa498ba4b21e24f92b8402e1bd064
70,70,0.904167,1344,rudy,fascio,r,wci,n/a,2.778238e+09,NaN,6692,rudolph,fascio,r,llead,new-orleans-pd,NaN,6aa0e3fdb9312c8f8aea1732ec85301e,03d61c017a3bdbcf2d8e843d0757eda3


In [3]:
# df.to_csv("../data/output/wcoi-llead.csv", index=False)